In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.model_selection import train_test_split
# Building a english to french translator

In [2]:
## Tutorial:
## https://medium.com/@dev.elect.iitd/neural-machine-translation-using-word-level-seq2seq-model-47538cba8cd7

In [3]:
df = pd.read_csv('action_state_text.csv', index_col=0)
df = df.iloc[:2000,:]
df

,action,state
0,hear argument morning case number brnovich ver...,mr chief justice may please court think key co...
1,mr carvin understand test articulate reduce an...,entirely mr chief justice reason involve diffe...
2,talk concern analysis would drive racial propo...,well mean neutral system must change order max...
3,really maximize participation equalize word co...,well example would eliminate valuable antifrau...
4,thank mr chief justice mr carvin understand ra...,well justice thomas think speak precisely term...
...,...,...
1995,guess i'm i'm ask second time around way word ...,example tell independent review do context dir...
1996,good morning general wall i'm go let talk meri...,well look founding think vattel quote good thi...
1997,undocumented person country say year know even...,take long-term embassy personnel somebody who'...
1998,mr palmore regardless whether zero understand ...,take long-term embassy personnel somebody who'...


In [4]:
all_action_words=set()
for a in df.action:
    for word in a.split():
        if word not in all_action_words:
            all_action_words.add(word)
    
all_state_words=set()
for s in df.state:
    for word in s.split():
        if word not in all_state_words:
            all_state_words.add(word)

In [5]:
len(all_action_words), len(all_state_words)

(4835, 6522)

In [6]:
lenght_list=[]
for l in df.state:
    lenght_list.append(len(l.split(' ')))
np.max(lenght_list)

331

In [7]:
lenght_list=[]
for l in df.action:
    lenght_list.append(len(l.split(' ')))
np.max(lenght_list)

201

In [8]:
input_words = sorted(list(all_state_words))
target_words = sorted(list(all_action_words))
num_encoder_tokens = len(all_state_words)
num_decoder_tokens = len(all_action_words)
# del all_eng_words, all_french_words

In [9]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

In [10]:
encoder_input_data = np.zeros(
    (len(df.state), 577),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(df.action), 217),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(df.action), 217, num_decoder_tokens),
    dtype='float32')

In [11]:
for i, (input_text, target_text) in enumerate(zip(df.state, df.action)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [12]:
print(encoder_input_data.shape, decoder_input_data.shape, decoder_target_data.shape)

(2000, 577) (2000, 217) (2000, 217, 4835)


In [13]:
embedding_size = 50

In [18]:
import keras
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model

In [15]:
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dex=  Embedding(num_decoder_tokens, embedding_size)
final_dex= dex(decoder_inputs)
decoder_lstm = LSTM(50, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [26]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [27]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     326100      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     241750      input_2[0][0]                    
_______________________________________________________________________________________

In [28]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=20,
          validation_split=0.05)

Epoch 1/20
15/15 [==============================] - 92s 6s/step - loss: 1.0768 - acc: 0.0013 - val_loss: 1.0314 - val_acc: 0.0014
Epoch 2/20
15/15 [==============================] - 99s 7s/step - loss: 0.9790 - acc: 0.0015 - val_loss: 0.9832 - val_acc: 0.0014
Epoch 3/20
15/15 [==============================] - 95s 6s/step - loss: 0.9451 - acc: 0.0015 - val_loss: 0.9708 - val_acc: 0.0014
Epoch 4/20
15/15 [==============================] - 101s 7s/step - loss: 0.9346 - acc: 0.0024 - val_loss: 0.9703 - val_acc: 0.0031
Epoch 5/20
15/15 [==============================] - 92s 6s/step - loss: 0.9319 - acc: 0.0027 - val_loss: 0.9730 - val_acc: 0.0031
Epoch 6/20
15/15 [==============================] - 90s 6s/step - loss: 0.9312 - acc: 0.0027 - val_loss: 0.9754 - val_acc: 0.0031
Epoch 7/20
15/15 [==============================] - 93s 6s/step - loss: 0.9306 - acc: 0.0027 - val_loss: 0.9771 - val_acc: 0.0031
Epoch 8/20
15/15 [==============================] - 91s 6s/step - loss: 0.9295 - acc: 0.0

In [32]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 50)          326100    
_________________________________________________________________
lstm (LSTM)                  [(None, 50), (None, 50),  20200     
Total params: 346,300
Trainable params: 346,300
Non-trainable params: 0
_________________________________________________________________


In [29]:
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [38]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
#     target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [42]:
for seq_index in [10,15,200,254,780]:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('------------------')
    print('Input sentence:', df.state[seq_index: seq_index + 1].values)
    print('Decoded sentence:', decoded_sentence)


------------------
Input sentence: ["think mean court meant crawford understand usual burden voting make good point know need benchmark usual burden otherwise meant congress invalidate virtually every time place manner restriction need safe harbor point we're making section immunize minority usual burden voting say show right precinct sort thing there's nothing language section somehow exempt long roughly commensurate normal election day system exist would constitute usual burden voting"]
Decoded sentence:  question question question question question question
------------------
Input sentence: ['would say equally open mean take account demographic reality one polling place five people one polling place million people obviously latter situation people equal opportunity vote']
Decoded sentence:  question question question question question question
------------------
Input sentence: ["well sort discretion say that's standard feature fee-shifting statute mean fee-shifting statute mean se

In [43]:
model.save("seq2seq_wordlevel")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: seq2seq_wordlevel/assets


In [19]:
model = keras.models.load_model('seq2seq_wordlevel')